# Grabbing data with cenpy

Cenpy (`sen - pie`) is a package that exposes APIs from the US Census Bureau and makes it easy to pull down and work with Census data in Pandas. First, notice that there are two core modules in the package, `base` and `explorer`, which each do different things. First, let's look at `explorer`. 

In [1]:
import cenpy.explorer as explorer
import pandas

On import, `explorer` requests all currently available APIs from the Census Bureau's [API listing](http://www.census.gov/data/developers/data-sets.html). In future, it will can also read a `JSON` collection describing the databases from disk, if asked.

Explorer has two functions, `available` and `explain`. `available` will provide a list of the identifiers of all the APIs that `cenpy` knows about. If run with `verbose=True`, `cenpy` will also include the title of the database as a dictionary. It's a good idea to *not* process this directly, and instead use it to explore currently available APIs. 

In [2]:
explorer.available(verbose=True)

{u'1990sf1': u'1990 Decennial: Summary File 1',
 u'2000sf1': u'2000 Decennial: Summary File 1',
 u'2000sf3': u'2000 Decennial: Summary File 3',
 u'2007ewks': u'2007 Economic Census - All Sectors: Economy-Wide Key Statistics',
 u'2010acs5': u'2010 American Community Survey: 5-Year Estimates',
 u'2010sf1': u'2010 Decennial: Summary File 1',
 u'2011acs5': u'2011 American Community Survey: 5-Year Estimates',
 u'2012acs1': u'2012 American Community Survey: 1-Year Estimates',
 u'2012acs1profile': u'2012 American Community Survey: 1-Year Profile Tables',
 u'2012acs3': u'2012 American Community Survey: 3-Year Estimates',
 u'2012acs3profile': u'2012 American Community Survey: 3-Year Profile Tables',
 u'2012acs5profile': u'2012 American Community Survey: 5-Year Profile Tables',
 u'2012ewks': u'2012 Economic Census - All Sectors: Economy-Wide Key Statistics',
 u'2012popproj/births': u'Vintage 2012 Population Projections - : Projected Births',
 u'2012popproj/deaths': u'Vintage 2012 Population Proj

The `explain` command provides the title and full description of the datasource. If run in verbose mode, the function returns the full `json` listing of the API. 

In [3]:
explorer.explain('2010acs5')

{u'2010 American Community Survey: 5-Year Estimates': u"The American Community Survey (ACS) is a nationwide survey designed to provide communities a fresh look at how they are changing. The ACS replaced the decennial census long form in 2010 and thereafter by collecting long form type information throughout the decade rather than only once every 10 years.  Questionnaires are mailed to a sample of addresses to obtain information about households -- that is, about each person and the housing unit itself.  The American Community Survey produces demographic, social, housing and economic estimates in the form of 1-year, 3-year and 5-year estimates based on population thresholds. The strength of the ACS is in estimating population and housing characteristics. It produces estimates for small areas, including census tracts and population subgroups.  Although the ACS produces population, demographic and housing unit estimates,it is the Census Bureau's Population Estimates Program that produces 

To actually connect to a database resource, you create a `Connection`. A `Connection` works like a *very* simplified connection from the `sqlalchemy` world. The `Connection` class has a method, `query` that constructs a query string and requests it from the Census server. This result is then parsed into JSON and returned to the user.  

In [4]:
import cenpy.base as base

In [5]:
conn = base.Connection('2010sf1')

In [6]:
conn

Connection to 2010 Decennial: Summary File 1 (ID: http://api.census.gov/data/id/2010sf1)

That may have taken longer than you'd've expected. This is because, when the `Connection` constructor is called, it populates the connection object with a bit of metadata that makes it possible to construct queries without referring to the census handbooks. 

For instance, a connection's `variables` represent all available search parameters for a given dataset. 

In [7]:
conn.variables.head()

,concept,label,predicateOnly,predicateType
AIANHH,Geographic Summary Level,GEO PLACE HOLDER,NaN,NaN
AIANHHCC,Geographic Characteristics,GEO PLACE HOLDER,NaN,NaN
AIANHHFP,Geographic Characteristics,GEO PLACE HOLDER,NaN,NaN
AIHHTLI,Geographic Characteristics,GEO PLACE HOLDER,NaN,NaN
AITS,Geographic Characteristics,GEO PLACE HOLDER,NaN,NaN


This dataframe is populated just like the census's table describing the variables on the corresponding [api website](http://api.census.gov/data/2010/sf1/variables.html). Fortunately, this means that you can modify and filter this dataframe just like you can regular pandas dataframes, so working out what the exact codes to use in your query is easy. 

In [8]:
filt = [True if 'B000' in x else False for x in conn.variables.index]

In [9]:
conn.variables[filt]

,concept,label,predicateOnly,predicateType
H011B0001,H11B. TOTAL POPULATION IN OCCUPIED HOUSING UNI...,Population in occupied housing units with a ho...,NaN,NaN
H011B0002,H11B. TOTAL POPULATION IN OCCUPIED HOUSING UNI...,Owned with a mortgage or a loan,NaN,NaN
H011B0003,H11B. TOTAL POPULATION IN OCCUPIED HOUSING UNI...,Owned free and clear,NaN,NaN
H011B0004,H11B. TOTAL POPULATION IN OCCUPIED HOUSING UNI...,Renter occupied,NaN,NaN
H012B0001,H12B. AVERAGE HOUSEHOLD SIZE OF OCCUPIED HOUSI...,Total,NaN,NaN
H012B0002,H12B. AVERAGE HOUSEHOLD SIZE OF OCCUPIED HOUSI...,Total !! Owner occupied,NaN,NaN
H012B0003,H12B. AVERAGE HOUSEHOLD SIZE OF OCCUPIED HOUSI...,Total !! Renter occupied,NaN,NaN
H016B0001,H16B. TENURE BY HOUSEHOLD SIZE (BLACK OR AFRIC...,Occupied housing units with a householder who ...,NaN,NaN
H016B0002,H16B. TENURE BY HOUSEHOLD SIZE (BLACK OR AFRIC...,Owner occupied:,NaN,NaN
H016B0003,H16B. TENURE BY HOUSEHOLD SIZE (BLACK OR AFRIC...,Owner occupied: !! 1-person household,NaN,NaN


Likewise, the different levels of geographic scale are determined from the metadata in the overall API listing and recorded. 

However, many Census products have multiple possible geographical indexing systems, like the deprecated `fips` code system and the new *Geographical Names Information System*, `gnis`. Thus, the `geographies` property is a dictionary of dataframes, where each key is the name of the identifier system and the value is the dataframe describing the identifier system. 

For the 2010 census, only `fips` and `gnis` systems are available. 

In [10]:
conn.geographies.keys()

[u'fips', u'gnis']

In [11]:
conn.geographies['fips']

,geoLevelId,name,optionalWithWCFor,requires
0,040,state,NaN,NaN
1,050,county,state,[state]
2,060,county subdivision,NaN,"[state, county]"
3,067,subminor civil subdivision,NaN,"[state, county, county subdivision]"
4,101,block,NaN,"[state, county, tract]"
5,140,tract,county,"[state, county]"
6,150,block group,tract,"[state, county, tract]"
7,160,place,state,[state]
8,230,alaska native regional corporation,state,[state]
9,280,american indian area/alaska native area/hawaii...,state,[state]


In [12]:
conn.geographies['gnis']

,geoLevelId,name
0,170,consolidated city
1,050,county
2,060,county subdivision
3,160,place
4,040,state
5,067,subminor civil subdivision


Note that some geographies in the `fips` system have a **required** filter to prevent drawing too much data. This will get passed to the `query` method later. 

So, let's just grab the housing information from the 2010 Census Short Form. Using the variables table above, we picked out a subset of the fields we wanted. Since the variables table is indexed by the identifiers, we can grab the indexes of the filtered dataframe as query parameters. 

In addition, adding the `NAME` field smart-fills the table with the name of the geographic entity being pulled from the Census.

In [13]:
cols = conn.variables[filt].index.tolist()

In [14]:
cols.append('NAME')

In [15]:
cols

[u'H011B0001',
 u'H011B0002',
 u'H011B0003',
 u'H011B0004',
 u'H012B0001',
 u'H012B0002',
 u'H012B0003',
 u'H016B0001',
 u'H016B0002',
 u'H016B0003',
 u'H016B0004',
 u'H016B0005',
 u'H016B0006',
 u'H016B0007',
 u'H016B0008',
 u'H016B0009',
 u'H017B0001',
 u'H017B0002',
 u'H017B0003',
 u'H017B0004',
 u'H017B0005',
 u'H017B0006',
 u'H017B0007',
 u'H017B0008',
 u'H017B0009',
 'NAME']

Now the query. The query is constructed just like the API query, and works as follows. 

1. cols - list of columns desired from the database, maps to census API's `get=`
2. geo_unit - string denoting the unit of study to pull, maps to census API's `in=`
3. geo_filter - dictionary containing groupings of geo_units, if required, maps to `for=`
    
To be specific, a fully query tells the server *what* columns to pull of *what* underlying geography from *what* aggregation units. It's structured using these heterogeneous datatypes so it's easy to change the smallest units quickly, while providing sufficient granularity to change the filters and columns as you go. 

This query below grabs the names, population, and housing estimates from the ACS, as well as their standard errors from census designated places in Arizona. 


In [16]:
data = conn.query(cols, geo_unit = 'place:*', geo_filter = {'state':'04'})

Once constructed, the query executes as fast as your internet connection will move. This query has:

In [17]:
data.shape

(451, 28)

28 columns and 451 rows. So, rather fast. 

For validity and ease of use, we store the last executed query to the object. If you're dodgy about your census API key never being shown in plaintext, never print this property!

In [18]:
conn.last_query

u'http://api.census.gov/data/2010/sf1?get=H011B0001,H011B0002,H011B0003,H011B0004,H012B0001,H012B0002,H012B0003,H016B0001,H016B0002,H016B0003,H016B0004,H016B0005,H016B0006,H016B0007,H016B0008,H016B0009,H017B0001,H017B0002,H017B0003,H017B0004,H017B0005,H017B0006,H017B0007,H017B0008,H017B0009,NAME&for=place:*&in=state:04+'

Then, since the results are returned as a pandas dataframe, we can easily process the results. First, it's helpful to know that pandas will not automatically infer numbers from the information returned. So, if you want to convert a numeric column, first mask the data and then cast the column to an integer. 

Using this, we can get the names and housing totals of the cities in Arizona with more than 5000 houses. 

In [19]:
data[data['H011B0001'].fillna(0).apply(int) > 5000][['NAME', 'H011B0001']]

,NAME,H011B0001
21,Avondale city,7271
63,Chandler city,11489
146,Gilbert town,7196
148,Glendale city,13535
224,Mesa city,15017
266,Peoria city,5201
268,Phoenix city,90934
366,Surprise city,6290
375,Tempe city,8753
394,Tucson city,23703


And, just in case you're liable to forget your FIPS codes, the explorer module can look up some fips codes listings for you.

In [20]:
explorer.fips_table('place', in_state='AZ')

reading http://www2.census.gov/geo/docs/reference/codes/files/st04_az_places.txt


,0,1,2,3,4,5,6
0,AZ,4,730,Aguila CDP,Census Designated Place,S,Maricopa County
1,AZ,4,870,Ajo CDP,Census Designated Place,S,Pima County
2,AZ,4,940,Ak Chin CDP,Census Designated Place,S,Pima County
3,AZ,4,1090,Ak-Chin Village CDP,Census Designated Place,S,Pinal County
4,AZ,4,1170,Alamo Lake CDP,Census Designated Place,S,La Paz County
5,AZ,4,1560,Ali Chuk CDP,Census Designated Place,S,Pima County
6,AZ,4,1570,Ali Chukson CDP,Census Designated Place,S,Pima County
7,AZ,4,1620,Ali Molina CDP,Census Designated Place,S,Pima County
8,AZ,4,1920,Alpine CDP,Census Designated Place,S,Apache County
9,AZ,4,1990,Amado CDP,Census Designated Place,S,Santa Cruz County


## OK, that's one API, does it work for others?

We'll try the Economic Census

In [21]:
conn2 = base.Connection('2012ewks')

Alright, let's look at the available columns:

In [22]:
filt2 = [True if 'Statistics' in x else False for x in conn2.variables['concept']]

In [23]:
conn2.variables[filt2]

,concept,label,predicateOnly,predicateType,required
EMP,Economic Statistics,Number of employees,NaN,string,NaN
ESTAB,Economic Statistics,Number of establishments,NaN,string,NaN
FOOTID_NAICS,Economic Statistics,FootID of NAICS,NaN,string,NaN
NAICS2012,Economic Statistics,NAICS2012 industry code,NaN,string,True
NAICS2012_TTL,Economic Statistics,Title of NAICS2012,NaN,string,NaN
NESTAB,Economic Statistics,Number of nonemployer establishments,NaN,string,NaN
NRCPTOT,Economic Statistics,"Nonemployer value of business done ($1,000)",NaN,string,NaN
OPTAX,Economic Statistics,Type of operation or tax status code,NaN,string,NaN
PAYANN,Economic Statistics,"Annual payroll ($1,000)",NaN,string,NaN
PAYQTR1,Economic Statistics,Total first quarter payroll,NaN,string,NaN


To show the required predicates, we can construct yet another filter. Note that *required* means that the query **will fail** if these are not passed as keyword arguments. They don't have to specify a single value, though, so they can be left as a wild card, like we did with `place:*` in the prior query:

In [24]:
filt3 = [True if x is True else False for x in conn2.variables['required']]

In [25]:
conn2.variables[filt3]

,concept,label,predicateOnly,predicateType,required
NAICS2012,Economic Statistics,NAICS2012 industry code,NaN,string,True


Like before, geographies are shown with their requirements. Here, the only geography is the `fips` geography. 

In [26]:
conn2.geographies.keys()

[u'fips']

In [27]:
conn2.geographies['fips']

,geoLevelId,name,requires
0,01,us,NaN
1,02,state,NaN
2,03,county,[state]


Now, we'll do some fun with error handling and passing of additional arguments to the query. Any "extra" required predicates beyond `get`, `for` and `in` are added at the end of the query as keyword arguments. These are caught and introduced into the query following the API specifications. 

First, though, let's see what happens when we submit a malformed query!

Here, we can query for every column in the dataset applied to places in California (`fips = 06`). The dataset we're working with, the Economic Census, requires an `OPTAX` field, which identifies the "type of operation or tax status code" along which to slice the data. Just like the other arguments, we will map them to keywords in the API string, and a wildcard represents a slice of all possible values. 

In [28]:
cols = [x for x in conn2.variables.index if 'ESTAB' in x]

In [29]:
data2 = conn2.query(cols=cols, geo_unit='county:*', geo_filter={'state':'06'})

HTTPError: 400 error: error: missing required variable/predicate: NAICS2012

So, the error provided from the Census is passed through. This will tell you if you're missing required predicates, or provided predicates are not understood. 

Fortunately, all APIs will tell you, from the `required` field in the `variables` table, what predicates are needed as keyword arguments and exactly what predicates are not understood. Here, we see that we need to have a NAICS2012 predicate to complete the query, which is not in the query string:

In [30]:
conn2.last_query

u'http://api.census.gov/data/2012/ewks?get=ESTAB,ESTAB_F,NESTAB,NESTAB_F&for=county:*&in=state:06+'

So, let's grab this data for any NAICS code! I've also stored my api key in a file called `api.key`. I'll read that in and use it for this query. 

In [31]:
apikey = open('./api.key').readlines()[0].strip('\n')

In [32]:
data2 = conn2.query(cols=cols, geo_unit='county:*', geo_filter={'state':'06'}, NAICS2012='*', key=apikey)

In [33]:
data2.head()

,ESTAB,ESTAB_F,NESTAB,NESTAB_F,NAICS2012,state,county
0,19,,38,,22,06,001
1,19,,38,,221,06,001
2,1846,,1690,,31-33,06,001
3,205,,197,,311,06,001
4,16,,11,,3113,06,001


And so you get the table of NAICS data for employment and establishments in places in California. 

## ToDo:


2. Provide joins with the geodata from the TIGER rest api